https://en.wikipedia.org/wiki/N-sphere#Spherical_coordinates

In [1]:
import numpy as np

def polar_to_cartesian(radius, angles):
    n = len(angles)+1
    cartesian_coordinates = np.ones(n) * radius
    for i in range(n-1):
        # print(i)
        for j in range(i):
            # print('sin ',j+1)
            cartesian_coordinates[i] *= np.sin(angles[j])
        # print('cos ',i+1)
        cartesian_coordinates[i] *= np.cos(angles[i])
    # print('sin ',n-1)
    cartesian_coordinates[n-1] *= np.prod([np.sin(angles[i]) for i in range(n-1)])
    return cartesian_coordinates

# Number of dimensions
n = 5

# Fixed radius
radius = 1

# Generate random angles in radians
theta = np.random.uniform(0, 2*np.pi, n)
print(theta)

# Convert to Cartesian coordinates with the given radius
cartesian_coordinates = polar_to_cartesian(radius, theta)

# Display the results
print("Generated angles in radians:", theta)
print(f"Converted Cartesian coordinates with radius {radius}:", cartesian_coordinates)

[5.18009163 5.83399049 5.41977889 1.37695042 0.46098187]
Generated angles in radians: [5.18009163 5.83399049 5.41977889 1.37695042 0.46098187]
Converted Cartesian coordinates with radius 1: [ 0.45083685 -0.80405717  0.25188642 -0.05675077 -0.25891019 -0.12859351]


In [2]:
import torch

In [3]:
radius = np.sqrt(2)
list_theta = []
# sampling random angles for the gaussian means
for i in range(2):
    list_theta.append(np.random.uniform(0, 2*np.pi, 2-1)) # -1 cause one dimension is fixed by the radius
list_theta = [np.array(theta) for theta in [[np.pi/4],[(3/2)*np.pi - np.pi/4]]]
gaussian_means = [torch.from_numpy(polar_to_cartesian(radius, theta)).float() for theta in list_theta]
print(gaussian_means)

[tensor([1., 1.]), tensor([-1., -1.])]


In [4]:
def get_difficulty_boundaries(radius, target_probability):
    return np.log((1-target_probability)/target_probability) + radius**2

In [7]:
get_difficulty_boundaries(radius, .9)

-0.19722457733621912

In [22]:
import random

In [38]:
n_difficulty_levels = 4
n_items = 10
difficulty_levels = np.linspace(
    get_difficulty_boundaries(radius, .95), # lower difficulty boundary
    get_difficulty_boundaries(radius, .3), # higher difficulty boundary
    n_difficulty_levels)  # number of difficulty levels
random.choices(
    population=list(difficulty_levels), 
    weights=None, # weights are all equal. However, we could use a different distribution
    k=n_items)

[-0.9444389791664389,
 -0.9444389791664389,
 0.3194733006847754,
 -0.9444389791664389,
 2.8472978603872043,
 2.8472978603872043,
 1.5833855805359898,
 0.3194733006847754,
 2.8472978603872043,
 0.3194733006847754]

In [36]:
difficulty_levels

array([-0.94443898,  0.3194733 ,  1.58338558,  2.84729786])

In [ ]:
get_difficulty_boundaries(radius, .3) for k in range(100)

In [2]:
from sklearn.metrics import confusion_matrix

y_true = [2, 0, 2, 2, 0, 1]
y_pred = [0, 0, 2, 2, 0, 2]

confusion_matrix(y_true, y_pred).tolist()

[[2, 0, 0], [0, 0, 1], [1, 0, 2]]